In [1]:
import $file.hw5stdlib
import hw5stdlib._

<console>: 89

In [2]:
// Nats

sealed trait Nat
case object Zero extends Nat
case class Succ(pred : Nat) extends Nat

val one = Succ(Zero)
val two = Succ(one)
val three = Succ(two)
val four = Succ(three)
val five = Succ(four)
val six = Succ(five)
val seven = Succ(six)
val eight = Succ(seven)
val nine = Succ(eight)
val ten = Succ(nine)

def nat_plus(x : Nat, y : Nat) : Nat = x match {
    case Zero    => y
    case Succ(x) => Succ(nat_plus(x, y))
}

def nat_to_int(x : Nat) : Int = x match {
    case Zero => 0
    case Succ( x ) => 1 + nat_to_int(x)
}

def nat_to_str(x : Nat) = nat_to_int(x).toString()

def print_nat(x : Nat) = println(nat_to_str(x))

def nat_mult(x : Nat, y : Nat) : Nat = x match {
    case Zero    => Zero
    case Succ(x) => nat_plus(nat_mult(x,y), y)
}


def nat_minus(x : Nat, y : Nat) : Nat = (x, y) match {
    case (Zero, _)          => Zero
    case (x, Zero)          => x
    case (Succ(x), Succ(y)) => nat_minus(x, y)
}

def nat_pow(x : Nat, y : Nat) : Nat = x match {
    case Zero       => one
    case Succ(Zero) => y 
    case Succ(x)    => nat_mult(y, nat_pow(x, y))
}

def nat_lte(x : Nat, y : Nat) : Bool = (x, y) match {
    case (Zero, y)          => True
    case (x, Zero)          => False
    case (Succ(x), Succ(y)) => nat_lte(x, y)
}

// Integers

sealed trait Integer
case class Positive(x : Nat) extends Integer
case class Negative(x : Nat) extends Integer

def int_to_str(x : Integer) : String = x match {
    case Positive(x) => nat_to_str(x)
    case Negative(x) => "-" + nat_to_str(x)
}

def print_integer(x : Integer) = println(int_to_str(x))

def abs(x : Integer) : Nat = x match {
    case Positive(x) => x
    case Negative(x) => x
}

def negate(x : Integer) : Integer = x match {
    case Positive(x) => Negative(x)
    case Negative(x) => Positive(x)
}

def plus(n : Integer, m : Integer) : Integer = (n, m) match {
    case (Positive(x), Positive(y)) => Positive(nat_plus(x, y))
    case (Negative(x), Negative(y)) => Negative(nat_plus(x,y))
    case (Negative(x), Positive(y)) => nat_lte(x, y) match {
        case True  => Positive(nat_minus(y, x))
        case False => Negative(nat_minus(x, y))
    }
    case (Positive(x), Negative(y)) => nat_lte(x, y) match {
        case True  => Negative(nat_minus(y, x))
        case False => Positive(nat_minus(x, y))
    }    
}

def minus(x : Integer, y : Integer) : Integer = plus(x, negate(y))

def mult(x : Integer, y : Integer) : Integer = (x,y) match {
    case (Positive(x), Positive(y)) => Positive(nat_mult(x, y))   
    case (Negative(x), Negative(y)) => Positive(nat_mult(x, y))
    case (Negative(x), Positive(y)) => Negative(nat_mult(x, y))
    case (Positive(x), Negative(y)) => Negative(nat_mult(x, y))
}

def pow(x : Integer, y : Nat) : Integer = y match {
    case Zero => Positive(Succ(Zero))
    case Succ(y) => mult(x, pow(x, y))
}

// Booleans

sealed trait Bool
case object True extends Bool
case object False extends Bool

def t = True
def f = False

def id(x : Bool) : Bool = x

def not(x : Bool) : Bool = x match {
    case True => False
    case False => True
}

def and(x : Bool, y : Bool) : Bool = (x,y) match {
    case (True, True) => True
    case _ => False
}

def or(x : Bool, y : Bool) : Bool = (x, y) match {
    case (False, False) => False
    case _              => True
}

def xor(x : Bool, y : Bool) : Bool = (x, y) match{
    case (True, False) => True
    case (False, True) => True
    case _ => False
}

def nand(x : Bool, y : Bool) : Bool = not(and(x,y))

def lte(x: Nat, y: Nat): Bool = (x, y) match {
    case (Zero, _) => True
    case (_, Zero) => False
    case (Succ(p1), Succ(p2)) => lte(p1, p2)
}

// Lists

sealed trait List[+A]
case object Empty extends List[Nothing]
case class Cons[A](x : A, xs : List[A]) extends List[A]

def length[A](xs : List[A]) : Nat = xs match {
    case Empty => Zero
    case Cons(_, xs) => Succ(length(xs))
}

def append[A](xs : List[A], ys : List[A]) : List[A] = xs match {
    case Empty => ys
    case Cons(x, xs) => Cons(x, append(xs, ys))
}

def fold[A, B](f : ((A, B) => B), acc : B, xs : List[A]) : B = xs match {
    case Empty       => acc
    case Cons(x, xs) => fold(f, f(x, acc), xs)
}

def map[A,B](f : (A => B), xs : List[A]) : List[B] = xs match {
    case Empty => Empty
    case Cons(x, xs) => Cons(f(x), map(f,xs))
}

def filter[A](p : (A => Bool), xs : List[A]) : List[A] = xs match {
    case Empty       => Empty
    case Cons(x, xs) => p(x) match {
        case True  => Cons(x, filter(p, xs))
        case False => filter(p, xs)
    }
}

def reverse[A](xs : List[A]) : List[A] =
    fold(Cons(_: A, _: List[A]), Empty, xs)

// Maybe
sealed trait Maybe[+A]
case object None extends Maybe[Nothing]
case class Just[A](x : A) extends Maybe[A]

def map_maybe[A,B](f : (A => B), mx : Maybe[A]) : Maybe[B] = mx match {
    case None => None
    case Just(x) => Just(f(x))
}

// Products

sealed trait Pair[+A, +B]
case class MkPair[A, B](fst : A, snd : B) extends Pair[A, B]

// Sums

sealed trait Either[+A, +B]
case class Left[A, B](left : A) extends Either[A, B]
case class Right[A, B](right : B) extends Either[A, B]

// Strings

def string_eq(s1 : String, s2 : String) : Bool = if (s1 == s2) True else False

// Equality
def bool_eq(x : Bool, y : Bool) : Bool = (x, y) match {
    case (True, True) => True
    case (False, False) => True
    case _ => False
}

def nat_eq(x : Nat, y : Nat) : Bool = (x, y) match {
    case (Zero, Zero)         => True
    case (Succ(px), Succ(py)) => nat_eq(px, py)
    case _                    => False
}

def int_eq(x : Integer, y : Integer) : Bool = (x, y) match {
    case (Positive(x), Positive(y))       => nat_eq(x, y)
    case (Negative(x), Negative(y))       => nat_eq(x, y)
    case (Positive(Zero), Negative(Zero)) => True
    case (Negative(Zero), Positive(Zero)) => True
    case _                                => False
}

def list_eq[A](l1: List[A], l2: List[A], eq: (A, A) => Bool): Bool =
    (l1, l2) match {
        case (Empty, Empty) => True
        case (Cons(x1, t1), Cons(x2, t2)) => and(eq(x1, x2), list_eq(t1, t2, eq))
        case _ => False
    }

// TEST HELPER
def passed(points: Int) {
    require(points >=0)
    if (points == 1) print(s"\n*** Tests Passed (1 point) ***\n")
    else print(s"\n*** Tests Passed ($points points) ***\n")
}

defined trait Nat
defined object Zero
defined class Succ
one: Succ = Succ(Zero)
two: Succ = Succ(Succ(Zero))
three: Succ = Succ(Succ(Succ(Zero)))
four: Succ = Succ(Succ(Succ(Succ(Zero))))
five: Succ = Succ(Succ(Succ(Succ(Succ(Zero)))))
six: Succ = Succ(Succ(Succ(Succ(Succ(Succ(Zero))))))
seven: Succ = Succ(Succ(Succ(Succ(Succ(Succ(Succ(Zero)))))))
eight: Succ = Succ(Succ(Succ(Succ(Succ(Succ(Succ(Succ(Zero))))))))
nine: Succ = Succ(Succ(Succ(Succ(Succ(Succ(Succ(Succ(Succ(Zero)))))))))
ten: Succ = Succ(Succ(Succ(Succ(Succ(Succ(Succ(Succ(Succ(Succ(Zero))))))))))
nat_plus: (x: Nat, y: Nat)Nat
nat_to_int: (x: Nat)Int
nat_to_str: (x: Nat)String
print_nat: (x: Nat)Unit
nat_mult: (x: Nat, y: Nat)Nat
nat_minus: (x: Nat, y: Nat)Nat
nat_pow: (x: Nat, y: Nat)Nat
nat_lte: (x: Nat, y: Nat)Bool
define...

# Homework 5

In this assignment we will develop some helpful structures for building the next interpreter.


## Dictionaries

We will need to define the datatype $\texttt{Dictionary}$ before we write the first interpreter for lettuce. We'll begin by giving the definition below:

$$
\begin{aligned}
\textbf{Dictionary}\ A\ B\ ::&= \text{EmptyDict} \\
&\mid\ \text{Entry}\ A\ B\ (\textbf{Dictionary}\ A\ B)
\end{aligned}
$$

This will create dictionaries of key-value pairs which will be very useful for implementing contexts in the next homework assignment. The definition of this type in Scala is given below:

In [3]:
sealed trait Dictionary[+A, +B] 
case object EmptyDict extends Dictionary[Nothing, Nothing]
case class Entry[A,B](key : A, value : B, xs : Dictionary[A, B]) extends Dictionary[A, B]

defined trait Dictionary
defined object EmptyDict
defined class Entry


For both of the following functions you will need to provide a parameter for an equality-checking function. This is so we can check if keys are equal to eachother

### Membership(In) (5pts)

Write a function called `isIn` which returns `True` if the Dictionary contains a given key and `False` if it does not.

In [4]:
def isIn[A,B](eq : ((A,A) => Bool), dict : Dictionary[A,B], y : A ) : Bool = dict match {
    case EmptyDict => False
    case Entry(x, value, xs) => eq(x, y) match {
                               case False => isIn(eq, xs, y)
                               case True => True
    }
}


isIn: [A, B](eq: (A, A) => Bool, dict: Dictionary[A,B], y: A)Bool


In [5]:
assert(isIn(nat_eq, Entry(five, True, EmptyDict), five) == True)
assert(isIn(nat_eq, Entry(eight, True, Entry(five, True, EmptyDict)), five) == True)
assert(isIn(nat_eq, Entry(five, True, EmptyDict), six) == False)
assert(isIn(nat_eq, EmptyDict, seven) == False)
passed(5)


*** Tests Passed (5 points) ***


### Lookup (5 pts)

Write a function that retrieves a value that corresponds to a key in a dictionary. If the key is not in the map then return Nothing.

$$
\texttt{lookup} : (a \rightarrow a \rightarrow \mathbb{B}) \rightarrow \texttt{Dict}\ a\ b \rightarrow a \rightarrow \texttt{Maybe}\ b
$$

In [6]:
def lookup[A,B](eq: (A,A)=>Bool, dict: Dictionary[A,B],y:A): Maybe[B]= dict match{
    case EmptyDict =>None
    case Entry(x,value,xs)=>eq(x,y) match{
                    case True=> Just(value)
                    case False=> lookup(eq, xs, y)
    }
}

lookup: [A, B](eq: (A, A) => Bool, dict: Dictionary[A,B], y: A)Maybe[B]


In [7]:
assert(lookup(nat_eq, Entry(five, "ham", EmptyDict), five) == Just("ham"))
assert(lookup(nat_eq, Entry(eight, seven, Entry(five, three, EmptyDict)), five) == Just(three))
assert(lookup(nat_eq, Entry(five, True, EmptyDict), six) == None)
assert(lookup(nat_eq, EmptyDict, seven) == None)
passed(5)


*** Tests Passed (5 points) ***


### Practice using these functions (5 pts)
#### Part A
Create the following dictionary as a scala value using this definition:
```
{
    five: True,
    six: False,
    seven: False
}
```
Place it in a variable named "`ans_3a`"

In [8]:
val ans_3a = Entry(five, True, Entry(six, False,Entry(seven,False, EmptyDict)))

Entry(Succ(Succ(Succ(Succ(Succ(Zero))))),True,Entry(Succ(Succ(Succ(Succ(Succ(Succ(Zero)))))),False,Entry(Succ(Succ(Succ(Succ(Succ(Succ(Succ(Zero))))))),False,EmptyDict)))

In [9]:
assert(lookup(nat_eq, ans_3a, five) == Just(True))
assert(lookup(nat_eq, ans_3a, six) == Just(False))
assert(lookup(nat_eq, ans_3a, seven) == Just(False))
passed(2)


*** Tests Passed (2 points) ***


#### Part B
For the following, use the super secret `secret_dict` variable defined below. Use the functions you just defined to complete the following:

1. Look up the value mapped by key `Positive(one)` and place it in variable `ans_3b1` 
2. Look up the value mapped by key `Positive(three)` and place it in variable `ans_3b2`
3. Check whether the key `Negative(three)` is in the map and place True or False in variable `ans_3b3`
4. Check whether the key `Negative(one)` is in the map and place True or False in variable `ans_3b4`


**Note**: For 1 and 2 remember to extract it from the maybe type, set the variable to the string `"does not exist"` if you get `None` instead of `Just(x)`.

In [10]:
val secret_dict = Entry(Positive(three), "abc",
                        Entry(Negative(one), "xyz",
                              Entry(Positive(one), "MIB", EmptyDict)))

Entry(Positive(Succ(Succ(Succ(Zero)))),abc,Entry(Negative(Succ(Zero)),xyz,Entry(Positive(Succ(Zero)),MIB,EmptyDict)))

In [11]:
val ans_3b1=lookup(int_eq,secret_dict,Positive(one))
val ans_3b2=lookup(int_eq,secret_dict,Positive(three))
val ans_3b3=isIn(int_eq,secret_dict,Negative(three))
val ans_3b4=isIn(int_eq,secret_dict,Negative(one))

True

In [12]:
// Hidden tests (3 pts)

input is incomplete: input is incomplete